In [ ]:
!nvidia-smi -L
import psutil
print(psutil.virtual_memory())

lemodel = 'mymodel'  # replace with new model name
leartist = 'unknown'
legenre = 'unknown'
leeventlog = False

In [ ]:
 from google.colab import drive
drive.mount('/content/gdrive')

<font color=purple size=5>💮 Setup 💮</font>

In [ ]:
!pip install git+https://github.com/openai/jukebox.git
!git clone https://github.com/openai/jukebox.git
!pip install av==8.1.0
!pip install tensorboardX
import os

lemodelpath = '/content/gdrive/MyDrive/juke/' + lemodel + '_prior/checkpoint_latest.pth.tar'
leaudio_files_dir = '/content/gdrive/MyDrive/' + lemodel
lepriorname = lemodel + '_prior'

print(lemodelpath,leaudio_files_dir,lepriorname,os.path.isfile(lemodelpath))

############### To Change to Google Drive

fin = open("/usr/local/lib/python3.8/dist-packages/jukebox/utils/logger.py", "rt")
data = fin.read()
data = data.replace('logdir = f"{hps.local_logdir}/{hps.name}"', 'logdir = f"/content/gdrive/MyDrive/juke/{hps.name}"')
fin.close()
fin = open("/usr/local/lib/python3.8/dist-packages/jukebox/utils/logger.py", "wt")
fin.write(data)
fin.close()

############### To Sample with Google Drive

fin = open("/usr/local/lib/python3.8/dist-packages/jukebox/hparams.py", "rt")
data = fin.read()
fin.close()

data += lemodel + """_prior = Hyperparams()
""" + lemodel + """_prior.update(prior_1b_lyrics)
""" + lemodel + """_prior.restore_prior='""" + lemodelpath + """'
""" + lemodel + """_prior.level=2
HPARAMS_REGISTRY['""" + lemodel + """_prior'] = """ + lemodel + """_prior
"""

#data = data.replace('y_bins=(10,100)','y_bins=(604, 7898)')
data = data.replace('min_duration=60.0','min_duration=24.0')
#data = data.replace('max_duration=600.0','max_duration=666.0')

fin = open("/usr/local/lib/python3.8/dist-packages/jukebox/hparams.py", "wt")
fin.write(data)
fin.close()

####################### To get Artist Genre, Lyrics

fin = open("/usr/local/lib/python3.8/dist-packages/jukebox/data/files_dataset.py", "rt")
data = fin.read()
fin.close()

data = data.replace('def get_metadata(self, filename, test):','''def get_metadata(self, filename, test):
        lyr = ""
        letx = filename.replace('.wav','.txt').replace('.mp3','.txt')
        print(letx)
        if os.path.isfile(letx):
          fin = open(letx, "rt")
          lyr = fin.read()
          print('yes')
          fin.close''')

data = data.replace('return None, None, None',"return '" + leartist + "', '" + legenre + "', lyr")
data = data.replace('import librosa', '''import librosa
import os''')


fin = open("/usr/local/lib/python3.8/dist-packages/jukebox/data/files_dataset.py", "wt")
fin.write(data)
fin.close()

####################### step print

fin = open("/content/jukebox/jukebox/train.py", "rt")
data = fin.read()
fin.close()

data = data.replace('log_inputs(orig_model, logger, x_in, y, x_out, hps)','log_inputs(orig_model, logger, x_in, y, x_out, hps);print(colored("steps:" + str(logger.iters),"magenta"))')
data = """from termcolor import colored

""" + data

if leeventlog == False:
  data = data.replace('logger.flush()','#logger.flush()')

fin = open("/content/jukebox/jukebox/train.py", "wt")
fin.write(data)
fin.close()


fin = open("/usr/local/lib/python3.8/dist-packages/jukebox/make_models.py", "rt")
data = fin.read(); fin.close()

data = data.replace("dist.barrier()","dist.barrier(); print('1')")
#data = data.replace("checkpoint = t.load(restore, map_location=t.device('cpu'))","checkpoint = t.load(restore, map_location=t.device('cuda')); print('2')")
data = data.replace(".format(restore))",".format(restore)); print(3); import gc; gc.collect()")

fin = open("/usr/local/lib/python3.8/dist-packages/jukebox/make_models.py", "wt")
fin.write(data); fin.close()


## Prior


In [ ]:
import gc; gc.collect()

lehps = ('vqvae,prior_1b_lyrics,all_fp16,cpu_ema','vqvae,prior_1b_lyrics,'+lepriorname+',all_fp16,cpu_ema')[os.path.isfile(lemodelpath)]
print(("new training model",lepriorname + " model found")[os.path.isfile(lemodelpath)])

!python jukebox/jukebox/train.py \
--prior --test --train --aug_shift --aug_blend \
--hps=$lehps \
--save_iters=238 \
--levels=3 \
--level=2 \
--name=$lepriorname \
--sample_length=786432     \
--bs=1 \
--audio_files_dir=$leaudio_files_dir  \
--weight_decay=0.01

